### About the dataset and clustering strategy

This dataset is about the neighborhoods in Toronto.   
In order to segment the neighborhoods and explore them, I will use the latitude and longitude coordinates of each neighborhood.  
I will scrape the neighborhoods from the Wikipedia page of the postal codes in Canada that start with M, which belong to Toronto: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

First, I will convert the neighborhoods into their equivalent latitude and longitude values via the Foursquare Geocode API.  
Afterwards, I will use the Foursquare API to explore most common venue categories in each neighborhood in Toronto.  
Afterwards, this feature will be used to group the neighborhoods into clusters.
The k-means clustering algorithm will be used to complete this task.  
Finally, the Folium library will be used to visualize the neighborhoods in Toronto and their emerging clusters.


### Scraping the data and loading it into a Pandas dataframe
First, I will scrape the data from Wikipedia using Python and the BeautifulSoup library.  
If we take a look at the HTML code of the Wikipedia page, all the required data is stored in a table class called **wikitable sortable**.  
Therefore, the strategy is to locate this wikipedia table and then iterate through all table rows that start with the **tr** tag.  
The values for postcode, borough and neighborhood that we are looking for are stored in the **td** tag of each table row **tr**.  
This data will be loaded into the Pandas dataframe neighborhoods:

In [151]:
# define the dataframe columns
column_names = ['Postalcode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# load libraries for scraping data
from bs4 import BeautifulSoup
import urllib.request
def make_soup(url):
    wikipage = urllib.request.urlopen(url)
    soupdata = BeautifulSoup(wikipage, "html.parser")
    return soupdata

soup = make_soup("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

# find the table
table = soup.find_all('table', class_="wikitable sortable")[0]  # Only use the first table

# iterate over it
wikidatasaved = []
for item in table.findAll('tr'):
    wikidata = []
    for data in item.findAll('td'):
        wikidata.append(data.text)
    if len(wikidata) == 3:
        neighborhoods = neighborhoods.append({'Postalcode': wikidata[0],
                                          'Borough': wikidata[1],
                                          'Neighborhood': wikidata[2]}, ignore_index=True)

# remove newline from Neighborhood column
neighborhoods['Neighborhood'] = neighborhoods['Neighborhood'].map(lambda x: str(x)[:-1])
neighborhoods.head()



,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Some postalcodes don't have a borough assigned to them.  
Let's drop these rows:

In [152]:
neighborhoods = neighborhoods[neighborhoods['Borough'] != 'Not assigned'].reset_index(drop = True)
neighborhoods.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Also, some postalcodes reference two different neighborhoods.  
Let's group these neighborhoods under one distinct postalcode:

In [153]:
neighborhoods = neighborhoods.groupby([neighborhoods.Postalcode, neighborhoods.Borough], as_index=False).agg(', '.join)

In [154]:
neighborhoods.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Finally, some boroughs don't have any neighborhood assigned to them.  
Let's find out which and use their borough name for their neighborhood classifier:

In [155]:
neighborhoods.loc[neighborhoods['Neighborhood'] == 'Not assigned']

,Postalcode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


In [156]:
neighborhoods['Neighborhood'].loc[neighborhoods['Neighborhood'] == 'Not assigned'] = neighborhoods['Borough']

Double-check that the Queen's Park borough now has a Neighborhood assigned:

In [157]:
neighborhoods[85:86]

,Postalcode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


Let's see how many distinct postal codes there are in Toronto:

In [158]:
neighborhoods.shape

(103, 3)

### Using Google geocode API to find out location data
Now that I have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name,   
I will use the Google geocode API to get the latitude and the longitude coordinates of each distinct postalcode in Toronto and join it with the previous dataframe:

In [168]:
import requests 
import json

# define the dataframe columns
column_names = ['Latitude', 'Longitude'] 

# instantiate the dataframe
locations = pd.DataFrame(columns=column_names)

apiKey = "REMOVED"
for index, row in neighborhoods.iterrows():
    address = row['Postalcode'] + ', Toronto'
    r = requests.get("https://maps.googleapis.com/maps/api/geocode/json?address="+ address +"&key="+ apiKey)
    location = json.loads(r.content.decode('utf-8'))
    if location['status'] != 'ZERO_RESULTS':
        locations = locations.append({'Latitude': location['results'][0]['geometry']['location']['lat'],
                                          'Longitude': location['results'][0]['geometry']['location']['lng'],}, ignore_index=True)
neighborhoods = neighborhoods.join(locations)
neighborhoods.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [268]:
print('The dataframe has {} boroughs, {} postalcodes and {} location data.'.format(
        len(neighborhoods['Borough'].unique()),
        len(neighborhoods['Postalcode'].unique()), len(neighborhoods['Latitude'].unique())
    )
)

The dataframe has 11 boroughs, 103 postalcodes and 103 location data.


### Create a folium map of Toronto and superimpose data points on top

The geopy geocode function will be quickly used get the central coordinates of Toronto, so that we can spread the boroughs and neighborhoods evenly around it later:


In [174]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [178]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   2.98 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  29.95 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  31.78 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  36.21 MB/s


### Use Foursquare API to explore and segment neighborhoods

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.  
Let's start off with defining the Foursquare credentials and the version:


In [179]:
CLIENT_ID = 'REMOVED' # your Foursquare ID
CLIENT_SECRET = 'REMOVED' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Let's take a look at the first postcode in our Toronto dataframe and get its coordinates:


In [180]:
neighborhoods.loc[0, 'Neighborhood']

'Rouge, Malvern'

In [182]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))



Latitude and longitude values of Rouge, Malvern are 43.8066863, -79.1943534.


Now, let's use the Foursquare API to get the top 100 venues that are in Toronto within a radius of 1000 meters from its center coordinates:

In [184]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results # display results

{'meta': {'code': 200, 'requestId': '5beeb07cdd57975cae3ac1f1'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4d669cba83865481c948fa53-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/spa_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1ed941735',
         'name': 'Spa',
         'pluralName': 'Spas',
         'primary': True,
         'shortName': 'Spa'}],
       'id': '4d669cba83865481c948fa53',
       'location': {'address': '8130 Sheppard Ave E',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside Ave',
        'distance': 595,
        'formattedAddress': ['8130 Sheppard Ave E (Morningside Ave)',
         'Toronto ON M1B 3W3',
         'Canada'],
        'labeledLatLngs': [{'label': 'd

Apparently, there are only 16 venues in a radius of 1000 meters of Rouge, Malvern.  
Let's see what kind of venues have been found by casting the JSON to a Pandas dataframe:

In [185]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [188]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON data into a Pandas dataframe

# filter columns 
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Tim Hortons,Coffee Shop,43.802000,-79.198169
2,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777
3,Wendy's,Fast Food Restaurant,43.802008,-79.198080
4,Wendy's,Fast Food Restaurant,43.807448,-79.199056


### Exploring Neighborhoods in Toronto
After exploring our data, let's create a function to do the same process to all neighborhoods in Toronto:

In [278]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [279]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The D

Let's check the size of the dataframe and the top 5 rows:

In [280]:
print(toronto_venues.shape)
toronto_venues.head()


(8502, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,African Rainforest Pavilion,43.817725,-79.183433,Zoo Exhibit
1,"Rouge, Malvern",43.806686,-79.194353,Toronto Pan Am Sports Centre,43.790623,-79.193869,Athletics & Sports
2,"Rouge, Malvern",43.806686,-79.194353,Canadiana exhibit,43.817962,-79.193374,Zoo Exhibit
3,"Rouge, Malvern",43.806686,-79.194353,Toronto Zoo,43.820582,-79.181551,Zoo
4,"Rouge, Malvern",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa


Let's see how many venues have been returned for each neighborhood given the radius of 1000 meters and the venue limit of 100:

In [281]:
toronto_venues.groupby('Neighborhood')['Neighborhood'].count()



Neighborhood
Adelaide, King, Richmond                                                                                         100
Agincourt                                                                                                        100
Agincourt North, L'Amoreaux East, Milliken, Steeles East                                                         100
Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown     43
Alderwood, Long Branch                                                                                           100
Bathurst Manor, Downsview North, Wilson Heights                                                                   52
Bayview Village                                                                                                   46
Bedford Park, Lawrence Manor East                                                                                100
Berczy Park                                        

Let's see how many unique venue categories have been collected in this dataframe:

In [282]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 329 uniques categories.


### Analyze each neighborhood

Now, each venue in a neighborhood will be sorted into its respective category.  
Afterwards, the mean frequency of occurence of a respective category in a neighborhood will be displayed.  
In essence, let's find out what kind of venues are in each neighborhood and how frequent these categories of venues are in regards to the total of venues.  
For example, if there are 5 restaurants with the category Italian restaurant in our collection of 100 venues, the category Italian restaurant will be assigned the value 0.05.  

First, let's use one-hot encoding to convert the categorical venue category to a binary classifier column for each venue category:

In [283]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhoods'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot

,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,...,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Let's see whether we have all 334 categories in the dataframe:

In [284]:
toronto_onehot.shape


(8502, 330)

Good.  
Let's group the rows by neighborhood and by taking the mean of the frequency of occurrence of each category as discussed in our strategy:

In [285]:
toronto_grouped = toronto_onehot.groupby('Neighborhoods').mean().reset_index()
toronto_grouped



,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,...,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,"Adelaide, King, Richmond",0.0,0.00,0.00,0.000000,0.00,0.00,0.020000,0.00,0.00,...,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Agincourt,0.0,0.00,0.00,0.000000,0.00,0.00,0.020000,0.00,0.00,...,0.00,0.00,0.0,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,...,0.00,0.00,0.0,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,...,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Alderwood, Long Branch",0.0,0.00,0.00,0.000000,0.00,0.00,0.010000,0.00,0.00,...,0.00,0.00,0.0,0.00,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.00,0.00,0.000000,0.00,0.00,0.019231,0.00,0.00,...,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Bayview Village,0.0,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,...,0.00,0.00,0.0,0.00,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Bedford Park, Lawrence Manor East",0.0,0.00,0.00,0.000000,0.00,0.00,0.010000,0.00,0.00,...,0.00,0.00,0.0,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000
8,Berczy Park,0.0,0.00,0.00,0.000000,0.00,0.00,0.020000,0.00,0.00,...,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Birch Cliff, Cliffside West",0.0,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,...,0.00,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Let's confirm whether we have all 103 postal code neighborhoods of Toronto and all venue categories in our dataframe:


In [287]:
toronto_grouped.shape


(103, 330)

Let's find out what the top 5 most common venues for each neighborhood in Toronto are:

In [288]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.06
1             Café  0.05
2            Hotel  0.04
3          Theater  0.04
4  Thai Restaurant  0.03


----Agincourt----
                 venue  freq
0   Chinese Restaurant  0.12
1          Coffee Shop  0.07
2             Pharmacy  0.05
3               Bakery  0.05
4  Japanese Restaurant  0.03


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                  venue  freq
0    Chinese Restaurant  0.20
1           Coffee Shop  0.07
2           Pizza Place  0.04
3          Noodle House  0.04
4  Cantonese Restaurant  0.03


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0           Coffee Shop  0.19
1     Indian Restaurant  0.12
2  Fast Food Restaurant  0.09
3           Pizza Place  0.09
4         Grocery Store  0.07


----Alderwood, Long Branch----
                  venue  freq
0           C

Great.  
Let's put this knowledge into a new dataframe with each neighborhood and the top 10 most common venues.  
First, let's write a function to sort the venues in descending order:

In [289]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [290]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Theater,Hotel,Steakhouse,Concert Hall,Thai Restaurant,Italian Restaurant,Gastropub,Sushi Restaurant
1,Agincourt,Chinese Restaurant,Coffee Shop,Bakery,Pharmacy,Restaurant,Indian Restaurant,Japanese Restaurant,Sandwich Place,Breakfast Spot,Gym / Fitness Center
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Coffee Shop,Noodle House,Pizza Place,Vietnamese Restaurant,Bakery,Tea Room,Dessert Shop,Korean Restaurant,Cantonese Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Coffee Shop,Indian Restaurant,Fast Food Restaurant,Pizza Place,Grocery Store,Sandwich Place,Steakhouse,Bank,Beer Store,Thrift / Vintage Store
4,"Alderwood, Long Branch",Coffee Shop,Fast Food Restaurant,Pizza Place,Breakfast Spot,Department Store,Burger Joint,Clothing Store,Seafood Restaurant,Sandwich Place,Café
5,"Bathurst Manor, Downsview North, Wilson Heights",Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Park,Convenience Store,Bank,Deli / Bodega,Restaurant,Bridal Shop
6,Bayview Village,Chinese Restaurant,Coffee Shop,Park,Pharmacy,Bank,Shopping Mall,Grocery Store,Café,Sushi Restaurant,Szechuan Restaurant
7,"Bedford Park, Lawrence Manor East",Coffee Shop,Bakery,Sushi Restaurant,Italian Restaurant,Sandwich Place,Pizza Place,Bagel Shop,Café,Grocery Store,Pharmacy
8,Berczy Park,Coffee Shop,Café,Italian Restaurant,Hotel,Park,Gastropub,Bakery,Steakhouse,Restaurant,Japanese Restaurant
9,"Birch Cliff, Cliffside West",Coffee Shop,Grocery Store,Bank,Park,Beer Store,Pizza Place,Golf Course,Pharmacy,Filipino Restaurant,Bar


### Cluster the neighborhoods in Toronto

Let's cluster the neighborhoods in 5 cluster.  
The k-means algorithm will be used for this:


In [291]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 1, 0, 1, 0, 1, 1, 2, 1], dtype=int32)

On first guessing, it looks like we found 3 main clusters and 2 outliers.  
Let's create a new dataframe that includes the clusters, as well as the top 10 venues for each neighborhood.

In [294]:
toronto_merged = neighborhoods

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoods'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2,Zoo Exhibit,Fast Food Restaurant,Pizza Place,Zoo,Athletics & Sports,Spa,Bus Station,Café,Caribbean Restaurant,Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,1,Coffee Shop,Pharmacy,Pizza Place,Sandwich Place,Breakfast Spot,Liquor Store,Bank,Fried Chicken Joint,Supermarket,Beer Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Pizza Place,Coffee Shop,Fast Food Restaurant,Pharmacy,Gym,Park,Fried Chicken Joint,Bank,Discount Store,Sandwich Place
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Fast Food Restaurant,Sandwich Place,Park,Grocery Store,Furniture / Home Store,Indian Restaurant,Supermarket,Beer Store,Chinese Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Coffee Shop,Clothing Store,Gym,Bank,American Restaurant,Fast Food Restaurant,Pizza Place,Restaurant,Indian Restaurant,Pharmacy


In [296]:

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters